# Point Cloud Demonstrations

## Registration

Registers two point clouds with slight offset.


In [1]:
import open3d as o3d
import numpy as np
import copy
import random
import copy
from collections import Counter

import utils
import correspondance as corr

In [ ]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

# load point clouds
base = o3d.io.read_point_cloud("./Point Clouds/Cleaner/1.ply")
source = o3d.io.read_point_cloud("./Point Clouds/Cleaner/8.ply")

# get 4x4 transformation matrix
trans = corr.register_clouds(base, source)

# show original clouds
o3d.visualization.draw_geometries([base, source])

# show aligned clouds
draw_registration_result_original_color(source, base, trans)


## Normals of point cloud
Shows the histogram of normals for the cloud. See utils module - running in notebook is much slower

## Outlier removal
Removes outliers and scatter artifacts from the pointcloud

In [ ]:
cl, ind=utils.clean_cloud("./Point Clouds/Cleaner/1.ply", view=True)

## Cleaned and combined 8 clouds

In [ ]:
base = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean_cropped.ply")
o3d.visualization.draw_geometries([base])

# Region Growing Segmentation
Demonstrates region growing algorithm for downsampled cloud

In [ ]:
cl = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean_cropped.ply")
pcd = cl.voxel_down_sample(0.01)

planes_list = corr.region_grow(pcd)

for plane in planes_list:
    colour = [random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)]
    for i in plane:
        pcd.colors[i] = colour

o3d.visualization.draw_geometries([pcd])

# extract walls which is the largest object in the cloud

walls = planes_list[np.argmax(np.array([len(i) for i in planes_list]))]
walls = pcd.select_down_sample(walls)

## Isolating each wall plane

In [ ]:
planes_list, plane_normals = corr.region_grow(walls, 0.8, True)

colours = [[1,0,0],[0,1,0],[0,0,1]]

for plane, colour in zip(planes_list, colours):
    for i in plane:
        walls.colors[i] = colour

o3d.visualization.draw_geometries([walls])

print(plane_normals)


## SVD of Walls
Principle vectors pick out the normals of the wall

In [ ]:
plane = planes_list[1]
wall_cp = copy.deepcopy(walls)
plane = wall_cp.select_down_sample(plane)
points = np.asarray(plane.points)
points -= np.mean(points, axis=0)
u, s, v = np.linalg.svd(points)

normal = np.matmul(np.linalg.pinv(points), np.ones(points.shape[0]))
normal /= np.linalg.norm(normal)

o3d.visualization.draw_geometries([plane, utils.create_vector_graph(v), utils.create_vector_graph([normal])])

## Match to plane in full size cloud  (deprec)

In [ ]:
# now match to the original full size point cloud

cl.paint_uniform_color([0.5,0.5,0.5])
print(plane_normals)
for norm, colour in zip(plane_normals, colours):
    dot_match = np.abs(1-np.asarray(cl.points).dot(norm))
    for i in range(len(dot_match)):
        if dot_match[i] <0.04:
            cl.colors[i] = colour

# add normals for clearer viewing
if not cl.has_normals():
    cl.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2, max_nn=30))
o3d.visualization.draw_geometries([cl])

## DBSCAN Methods (deprec)

In [ ]:
pcd = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean_cropped.ply")
pcd = pcd.voxel_down_sample(0.005)

isolated= corr.remove_planes(pcd)

labels = np.array(isolated.cluster_dbscan(eps=0.01, min_points=40, print_progress=True))
colours = np.random.rand(labels.max()+1, 3)
for i in range(len(labels)):
    if labels[i] == -1:
        isolated.colors[i] = [0,0,0]
        continue
    isolated.colors[i] = colours[labels[i]]
outliers = np.where(labels == -1)[0]
isolated = isolated.select_down_sample(outliers, invert=True)

o3d.visualization.draw_geometries([isolated])


## Merging model clouds


In [2]:
scale = 1

cl1 = o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Commander/1.3.ply")
cl1 = corr.isolate_model(cl1)
plane1 = utils.create_origin_plane(100)
cl1 += plane1
cl1.scale(scale)

for i in range(4):
    print(i)
    cl2 = o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Commander/1.{}.ply".format(i+2))
    cl2 = corr.isolate_model(cl2)
    temp_cl = cl2 + plane1
    
    temp_cl.scale(scale)
    cl2.scale(scale)
    print("Registering")
    tran = corr.register_clouds(cl1, temp_cl, voxel_radius=[0.01, 0.005, 0.0001], max_iter=[1000,1000,10000])
    
    cl2 = cl2.transform(tran)
    cl1 += cl2
    cl1.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2, max_nn=30))
    
o3d.visualization.draw_geometries([cl1])

  0%|          | 0/2477 [00:00<?, ?it/s]

0


100%|██████████| 2477/2477 [00:00<00:00, 15744.09it/s]


Registering


 94%|█████████▍| 2392/2546 [00:00<00:00, 18220.74it/s]

...

Registering


  0%|          | 0/2499 [00:00<?, ?it/s]

3


100%|██████████| 2499/2499 [00:00<00:00, 13873.77it/s]


Registering


# Reference models

In [ ]:
cmdr = o3d.io.read_point_cloud("./Point Clouds/Commander Ref.ply")
brdsd = o3d.io.read_point_cloud("./Point Clouds/Broadside Ref.ply")
o3d.visualization.draw_geometries([cmdr])
o3d.visualization.draw_geometries([brdsd])

# Recognising Models

In [3]:
isolated = o3d.io.read_point_cloud("isolated.ply")
o3d.visualization.draw_geometries([isolated])

labels = np.array(isolated.cluster_dbscan(eps=0.01, min_points=40, print_progress=True))
targets = []
print(Counter(labels))
for i in range(labels.max()+1):
    obj = np.where(labels==i)[0]
    obj = isolated.select_down_sample(obj)
    obj = obj.translate(-obj.get_center())
    targets.append(obj)
    print(len(obj.points))
    print(obj.get_oriented_bounding_box().volume())

Counter({1: 1690, 0: 1558, -1: 494, 3: 370, 2: 293})
1558
0.0001939184216741835
1690
0.0002557404839896712
293
2.5114991737324598e-05
370
3.814884067853866e-05


In [ ]:
# for i in targets:
#     print(i.get_oriented_bounding_box().volume())
cmdr = o3d.io.read_point_cloud("./Point Clouds/Commander Ref.ply")
brdsd = o3d.io.read_point_cloud("./Point Clouds/Broadside Ref.ply")

cmdr_vol = cmdr.get_oriented_bounding_box().volume()
brdsd_vol = brdsd.get_oriented_bounding_box().volume()
print("Ref volumes:")
print("Commander: {}".format(cmdr.get_oriented_bounding_box().volume()))
print("Broadside: {}".format(brdsd.get_oriented_bounding_box().volume()))
print(np.asarray(cmdr.get_oriented_bounding_box().get_box_points()))

# print("Commander matching")
# for pcd in targets:
#     vol = pcd.get_oriented_bounding_box().volume()
#     if vol > 0.3*cmdr_vol and vol < cmdr_vol:
#         o3d.visualization.draw_geometries([pcd])
        
# print("Broadside matching")
# for pcd in targets:
#     vol = pcd.get_oriented_bounding_box().volume()
#     if vol > 0.3*brdsd_vol and vol < brdsd_vol:
#         o3d.visualization.draw_geometries([pcd])